1. Cargar librerías y el dataset

In [1]:
import pandas as pd

df = pd.read_csv('../horarios-webscraper/combined_horarios.csv')
df.head()


,curso,nrc,id_liga,docente,dia,hora_inicio,hora_fin
0,CIEN-648,9441,T1,MURGA TORRES EMZON ENRIQUE,MIE,08:50:00,09:40:00
1,CIEN-648,9441,T1,MURGA TORRES EMZON ENRIQUE,MIE,09:45:00,10:35:00
2,CIEN-648,9442,P1,MURGA TORRES EMZON ENRIQUE,MIE,10:40:00,11:30:00
3,CIEN-648,9442,P1,MURGA TORRES EMZON ENRIQUE,MIE,11:35:00,12:25:00
4,CIEN-648,9443,L1,COSAVALENTE CULQUICHICON PAUL,MIE,13:25:00,14:15:00


2. Revisar valores nulos y duplicados

In [2]:
# Revisar valores nulos
print("Valores nulos por columna:")
print(df.isnull().sum())

# Revisar duplicados
print("\nNúmero de filas duplicadas:")
print(df.duplicated().sum())


Valores nulos por columna:
curso           0
nrc             0
id_liga        20
docente         0
dia             0
hora_inicio     0
hora_fin        0
dtype: int64

Número de filas duplicadas:
8


3. Eliminar duplicados y nulos

In [3]:
df = df.drop_duplicates()

In [6]:
df['id_liga'] = df['id_liga'].fillna('SIN_ID')


In [7]:
print("Filas con valores nulos:", df.isnull().sum())
print("Total de filas con al menos un nulo:", df.isnull().any(axis=1).sum())


Filas con valores nulos: curso          0
nrc            0
id_liga        0
docente        0
dia            0
hora_inicio    0
hora_fin       0
dtype: int64
Total de filas con al menos un nulo: 0


4. Validación de datos

In [8]:
# Revisar tipos de datos
print("Tipos de datos:")
print(df.dtypes)


Tipos de datos:
curso          object
nrc             int64
id_liga        object
docente        object
dia            object
hora_inicio    object
hora_fin       object
dtype: object


5. Transformación de columnas de tiempo

In [9]:
import pandas as pd

# Convertir a datetime solo la hora (sin fecha)
df['hora_inicio'] = pd.to_datetime(df['hora_inicio'], format='%H:%M:%S').dt.time
df['hora_fin'] = pd.to_datetime(df['hora_fin'], format='%H:%M:%S').dt.time


In [10]:
df.head()

,curso,nrc,id_liga,docente,dia,hora_inicio,hora_fin
0,CIEN-648,9441,T1,MURGA TORRES EMZON ENRIQUE,MIE,08:50:00,09:40:00
1,CIEN-648,9441,T1,MURGA TORRES EMZON ENRIQUE,MIE,09:45:00,10:35:00
2,CIEN-648,9442,P1,MURGA TORRES EMZON ENRIQUE,MIE,10:40:00,11:30:00
3,CIEN-648,9442,P1,MURGA TORRES EMZON ENRIQUE,MIE,11:35:00,12:25:00
4,CIEN-648,9443,L1,COSAVALENTE CULQUICHICON PAUL,MIE,13:25:00,14:15:00


 2. Normalización
 
No es necesaria por ahora porque nuestro dataset no tiene columnas numéricas continuas como notas, duración, etc.
Pero si creamos una columna con la duración de clase, ahí sí podríamos normalizarla (lo hacemos en la parte de ingeniería si aplica).

6. Ingeniería de características básica: Duración de clase

In [11]:
# Convertir a datetime completo para poder restar
df['inicio_dt'] = pd.to_datetime(df['hora_inicio'].astype(str))
df['fin_dt'] = pd.to_datetime(df['hora_fin'].astype(str))

# Calcular duración
df['duracion_min'] = (df['fin_dt'] - df['inicio_dt']).dt.total_seconds() / 60

# Eliminar columnas auxiliares
df = df.drop(columns=['inicio_dt', 'fin_dt'])


C:\Users\Administrador\AppData\Local\Temp\ipykernel_23180\2884609049.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['inicio_dt'] = pd.to_datetime(df['hora_inicio'].astype(str))
C:\Users\Administrador\AppData\Local\Temp\ipykernel_23180\2884609049.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['fin_dt'] = pd.to_datetime(df['hora_fin'].astype(str))


In [12]:
df.head()

,curso,nrc,id_liga,docente,dia,hora_inicio,hora_fin,duracion_min
0,CIEN-648,9441,T1,MURGA TORRES EMZON ENRIQUE,MIE,08:50:00,09:40:00,50.0
1,CIEN-648,9441,T1,MURGA TORRES EMZON ENRIQUE,MIE,09:45:00,10:35:00,50.0
2,CIEN-648,9442,P1,MURGA TORRES EMZON ENRIQUE,MIE,10:40:00,11:30:00,50.0
3,CIEN-648,9442,P1,MURGA TORRES EMZON ENRIQUE,MIE,11:35:00,12:25:00,50.0
4,CIEN-648,9443,L1,COSAVALENTE CULQUICHICON PAUL,MIE,13:25:00,14:15:00,50.0


7. Codificar columnas categóricas

In [15]:
# Codificar días (LUN=0, ..., SAB=5)
dias_map = {'LUN': 0, 'MAR': 1, 'MIE': 2, 'JUE': 3, 'VIE': 4, 'SAB': 5}
df['dia'] = df['dia'].map(dias_map)


In [18]:
df

,curso,nrc,id_liga,docente,dia,hora_inicio,hora_fin,duracion_min
0,CIEN-648,9441,T1,MURGA TORRES EMZON ENRIQUE,2,08:50:00,09:40:00,50.0
1,CIEN-648,9441,T1,MURGA TORRES EMZON ENRIQUE,2,09:45:00,10:35:00,50.0
2,CIEN-648,9442,P1,MURGA TORRES EMZON ENRIQUE,2,10:40:00,11:30:00,50.0
3,CIEN-648,9442,P1,MURGA TORRES EMZON ENRIQUE,2,11:35:00,12:25:00,50.0
4,CIEN-648,9443,L1,COSAVALENTE CULQUICHICON PAUL,2,13:25:00,14:15:00,50.0
...,...,...,...,...,...,...,...,...
299,ISIA-130,9691,P1,VASQUEZ PEREYRA JOSE HUMBERTO,1,09:45:00,10:35:00,50.0
300,ISIA-130,9692,L1,VASQUEZ PEREYRA JOSE HUMBERTO,4,07:00:00,07:50:00,50.0
301,ISIA-130,9692,L1,VASQUEZ PEREYRA JOSE HUMBERTO,4,07:55:00,08:45:00,50.0
302,ISIA-130,9692,L1,VASQUEZ PEREYRA JOSE HUMBERTO,4,08:50:00,09:40:00,50.0


8. Restablecer los índices para que estén en secuencia

In [ ]:
df = df.reset_index(drop=True)

df

,curso,nrc,id_liga,docente,dia,hora_inicio,hora_fin,duracion_min
0,CIEN-648,9441,T1,MURGA TORRES EMZON ENRIQUE,2,08:50:00,09:40:00,50.0
1,CIEN-648,9441,T1,MURGA TORRES EMZON ENRIQUE,2,09:45:00,10:35:00,50.0
2,CIEN-648,9442,P1,MURGA TORRES EMZON ENRIQUE,2,10:40:00,11:30:00,50.0
3,CIEN-648,9442,P1,MURGA TORRES EMZON ENRIQUE,2,11:35:00,12:25:00,50.0
4,CIEN-648,9443,L1,COSAVALENTE CULQUICHICON PAUL,2,13:25:00,14:15:00,50.0
...,...,...,...,...,...,...,...,...
291,ISIA-130,9691,P1,VASQUEZ PEREYRA JOSE HUMBERTO,1,09:45:00,10:35:00,50.0
292,ISIA-130,9692,L1,VASQUEZ PEREYRA JOSE HUMBERTO,4,07:00:00,07:50:00,50.0
293,ISIA-130,9692,L1,VASQUEZ PEREYRA JOSE HUMBERTO,4,07:55:00,08:45:00,50.0
294,ISIA-130,9692,L1,VASQUEZ PEREYRA JOSE HUMBERTO,4,08:50:00,09:40:00,50.0
